# 2. Subset Sum Problem Example
### James Saslow
### 12/3/2023

### The Subset Sum Problem (SSP) is a math problem that can be casted in the form of a LUBO.

SSP is an NP-Hard Problem where given some multiset $\mathscr{S}$ the question is to decide if there are any subset of integers that sum to the target $T$.

In this example, I will change the problem statement to make it a bit less arbitrary and arguably more practical.

So, my modified SSP is given some set $\mathscr{S} = \{s_{0}, s_{1}, ..., s_{N-1}\}$ where $s_{i} > 0$ find the subset of $\mathscr{S}$ whose sum returns the extrema of $T$.

This is example is a bit trivial since the entire set of positive integers sums to the maximum value that $T$ can be. Or in quantum computing language, the maximum solution is the state of all ones $|11...1>$. A '$1$' in this instance tells us to include $s_{i}$ in the sum, and a '$0$' tells us *not* to include $s_{i}$ in our sum. Thus $|11...1>$ tells us to sum everything in $\mathscr{S}$ together.

If we want to choose a subset of $\mathscr{S}$ that sums to the minimum value of $T$, then the solution is to not sum anything. This means that the minimum solution is the state of all zeros $|00...0>$

### Let's Define a Particular Example of the Modified SSP

Let's say I have a set $\mathscr{S} = \{1,2,3,4,5,6,7,8\}$

and I want to choose 
- 1) a subset of $\mathscr{S}$ that, when summed, is a minimum
- 2) a subset of $\mathscr{S}$ that, when summed, is a maximum

The solutions we expect are 
- 1) $X_{min} = |00000000>$
- 2) $X_{max} = |11111111>$

### Encoding and Solving Task 1 on DWave: The Minimum to the Modified SSP

In [1]:
# ================== Task 1: Finding the Minimum to the modified SSP =======================

# Importing Packages
import numpy as np

# DWave Packages
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

In [2]:
# Defining the QUBO parameters

items   = np.arange(0,8).tolist()#[0,1,2,3,4,5,6,7]  # Binary Variable Labeling
print(items)

weights = [1,2,3,4,5,6,7,8] # W weights associated with each binary variable

x = [f'Item_{i}' for i in items] # More Binary Variable Labeling

print(x)

[0, 1, 2, 3, 4, 5, 6, 7]
['Item_0', 'Item_1', 'Item_2', 'Item_3', 'Item_4', 'Item_5', 'Item_6', 'Item_7']


In [3]:
# Encoding the QUBO in DWave solvers

bqm = BinaryQuadraticModel("BINARY") # Encoding QUBO

# Writing the Objective Function (adding each variable and assigning it's associated weight)
for i in items:
    bqm.add_variable(x[i], weights[i])
    
    
bqm

BinaryQuadraticModel({'Item_0': 1.0, 'Item_1': 2.0, 'Item_2': 3.0, 'Item_3': 4.0, 'Item_4': 5.0, 'Item_5': 6.0, 'Item_6': 7.0, 'Item_7': 8.0}, {}, 0.0, 'BINARY')

In [4]:
# Solve the problem using D-Wave
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=50)

In [5]:
# Post-Processing Solver Output
opt_solution = sampleset.first.sample # Binary Bit-String that minimizes the cost function
opt_value = sampleset.first.energy    # Minimized Cost function value

In [7]:
print('X_min = ', opt_solution)
print('C_min = ', opt_value)

X_min =  {'Item_0': 0, 'Item_1': 0, 'Item_2': 0, 'Item_3': 0, 'Item_4': 0, 'Item_5': 0, 'Item_6': 0, 'Item_7': 0}
C_min =  0.0


### Minimum solution of the Modified SSP

After running this problem on DWave, we find that 

### $X_{min} = |00000000>$ , $C(X_{min}) = 0$

which is what we predicted!

Now, we need to use the $C(X) \rightarrow{} -C'(X) $ reflection trick to formulate $C'(X)$ to get the maximum of $C(X)$.

### Encoding and Solving Task 2 on DWave: The Maximum to the Modified SSP

In [8]:
# Defining the QUBO parameters

items   = np.arange(0,8).tolist()#[0,1,2,3,4,5,6,7]  # Binary Variable Labeling

# Just making all the weights negative
weights = [-1,-2,-3,-4,-5,-6,-7,-8] # W weights associated with each binary variable

x = [f'Item_{i}' for i in items] # More Binary Variable Labeling

In [9]:
# Encoding the QUBO in DWave solvers

bqm = BinaryQuadraticModel("BINARY") # Encoding QUBO

# Writing the Objective Function (adding each variable and assigning it's associated weight)
for i in items:
    bqm.add_variable(x[i], weights[i])
    
    
bqm

BinaryQuadraticModel({'Item_0': -1.0, 'Item_1': -2.0, 'Item_2': -3.0, 'Item_3': -4.0, 'Item_4': -5.0, 'Item_5': -6.0, 'Item_6': -7.0, 'Item_7': -8.0}, {}, 0.0, 'BINARY')

In [10]:
# Solve the problem using D-Wave
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=50)

In [11]:
# Post-Processing Solver Output
opt_solution = sampleset.first.sample # Binary Bit-String that minimizes the cost function
opt_value = sampleset.first.energy    # Minimized Cost function value

In [12]:
print('X_max = ', opt_solution)
print('C_max = ', -opt_value)

X_max =  {'Item_0': 1, 'Item_1': 1, 'Item_2': 1, 'Item_3': 1, 'Item_4': 1, 'Item_5': 1, 'Item_6': 1, 'Item_7': 1}
C_max =  36.0


### Maximum solution of the modified SSP

After running this problem on DWave, we find that

### $X_{max} = |11111111>$ , $C_{max} = 36$

which is also what we predicted!

Stay tuned for the next example where we investigate the *knapsack problem*,which is another LUBO problem similar to the modified subset sum problem, but with a constraint!

### Sources

[1] “Dynamic programming - subset sum problem,” GeeksforGeeks, https://www.geeksforgeeks.org/subset-sum-problem-dp-25/ (accessed Dec. 11, 2023). 

[2] “Binary quadratic models#,” Binary Quadratic Models - Ocean Documentation 6.7.1 documentation, https://docs.ocean.dwavesys.com/en/stable/concepts/bqm.html (accessed Dec. 11, 2023). 

[3] “Quantum programming tutorial | D-wave qubits 2021,” YouTube, https://www.youtube.com/watch?v=jTDnGox0c9Y (accessed Dec. 11, 2023). 